In [1]:
from collections import defaultdict
import re
import time
from unicodedata import normalize
import math

inverted_index = defaultdict(set)

def preprocess(text):
    text = text.lower()
    tokens = re.findall(r'\w+', text)
    return set(tokens)
    
def elimina_tildes(cadena):
    cadena = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", cadena), 0, re.I
    )
    cadena = normalize( 'NFC', cadena)
    return cadena
    
def createInvertedIndex():
    with open('/kaggle/input/dataset/01_corpus_turismo.txt') as documents:
        for doc_id, doc_text in enumerate(documents):
            doc_text = elimina_tildes(doc_text)
            terms = preprocess(doc_text)
            for term in terms:
                inverted_index[term].add(doc_id)
                
createInvertedIndex()
sorted_inverted_index = sorted(inverted_index)
print('Total de términos: ' + str(len(sorted_inverted_index)))
print(list(sorted_inverted_index))

Total de términos: 145
['2000', 'a', 'agua', 'alrededor', 'altitud', 'amazonia', 'andes', 'arquitectura', 'artesanal', 'ascensiones', 'atrae', 'atraen', 'aventura', 'aves', 'avistamiento', 'baños', 'bienestar', 'biodiversidad', 'bosques', 'cada', 'cajas', 'caminatas', 'canopy', 'cascadas', 'centro', 'cercania', 'ciudad', 'colonial', 'color', 'como', 'comunidades', 'con', 'conecta', 'conocer', 'conocido', 'cotopaxi', 'crater', 'cuenca', 'cuenta', 'cultura', 'de', 'declarado', 'del', 'deportes', 'deslumbra', 'disfrutan', 'durante', 'ecologico', 'ecuador', 'ecuatorianos', 'el', 'en', 'entre', 'es', 'excursionistas', 'exploracion', 'famosas', 'famoso', 'feriado', 'feriados', 'fiestas', 'galapagos', 'gastronomia', 'gran', 'guayaquil', 'historico', 'humanidad', 'ideales', 'impacta', 'impresionantes', 'indigena', 'indigenas', 'interesados', 'invita', 'islas', 'jovenes', 'la', 'laguna', 'lagunas', 'las', 'llena', 'locales', 'longevidad', 'lopez', 'los', 'malecon', 'mas', 'megadiverso', 'mercad

In [2]:
def query(query_text):
    start_time = time.time()
    terms = preprocess(query_text)
    results = set(doc_id for term in terms for doc_id in inverted_index.get(term, []))
    return results

# Headers
headers = ['Término']
for doc in range(16):
    headers.append('D' + str(doc + 1))
headers.append('IDF')

# Data
dataset = []
for term in sorted_inverted_index:
    data = []
    data.append(term)
    docs = query(term)
    for doc in range(16):
        data.append(1 if doc in docs else 0)

    # Cálculo de idf
    N = 16  # total documentos
    df_t = len(docs)  # documentos donde aparece el término

    idf = round(math.log10(N / df_t), 3)
    data.append(idf)
    dataset.append(data)

In [3]:
from tabulate import tabulate

table = tabulate(
    dataset,
    headers=headers,
    tablefmt="fancy_grid",
)

print(table)

╒════════════════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤═══════╤═══════╤═══════╤═══════╤═══════╤═══════╤═══════╤═══════╕
│ Término        │   D1 │   D2 │   D3 │   D4 │   D5 │   D6 │   D7 │   D8 │   D9 │   D10 │   D11 │   D12 │   D13 │   D14 │   D15 │   D16 │   IDF │
╞════════════════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╡
│ 2000           │    0 │    0 │    0 │    0 │    0 │    0 │    0 │    0 │    0 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │ 1.204 │
├────────────────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ a              │    0 │    0 │    0 │    0 │    0 │    0 │    1 │    1 │    0 │     1 │     0 │     0 │     1 │     0 │     0 │     0 │ 0.602 │
├────────────────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┼───────┼───────┼───────┼───────┼───